In [2]:
from urllib import request

import tensorflow as tf

from io import StringIO
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
from __future__ import print_function
from keras.layers import Dense, Activation, SimpleRNN, LSTM
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
import wordcloud as w
import matplotlib.pyplot as plot
import torch
from transformers import  BartTokenizer, TFBartForConditionalGeneration

2024-11-21 08:12:51.694507: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 08:12:51.703678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732169571.715038    7775 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732169571.718353    7775 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 08:12:51.729926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
shelley_frank = request.urlopen('https://www.gutenberg.org/cache/epub/84/pg84.txt').read()

In [4]:
shelley_frank = shelley_frank.decode('utf-8-sig')
print(shelley_frank[:100])

The Project Gutenberg eBook of Frankenstein; Or, The Modern Prometheus
    
This ebook is for the 


In [5]:
actual_start = 'You will rejoice'
word_index = shelley_frank.find(actual_start)

In [6]:
if word_index != -1:
    shelley_frank = shelley_frank[word_index:]

In [7]:
shelley_frank = re.sub(r'—', ' ', shelley_frank.lower())
shelley_frank = shelley_frank.replace('project gutenberg', '')
shelley_frank = nltk.word_tokenize(shelley_frank)
shelley_frank = ' '.join(shelley_frank)  

# Ensure space after punctuation if it's missing
print(shelley_frank[:100])

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which yo


In [8]:
shelley_frank = re.sub(r'[^a-zA-Z.,!?; ]', '', shelley_frank)
shelley_frank = re.sub(r'([.,!?;])', r'\1 ', shelley_frank)
shelley_frank = re.sub(r'\s+([.,!?;])', r'\1', shelley_frank)
shelley_frank = re.sub(r'\s+', ' ', shelley_frank) 
print(shelley_frank[:1000])

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings. i arrived here yesterday, and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking. i am already far north of london, and as i walk in the streets of petersburgh, i feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. do you understand this feeling? this breeze, which has travelled from the regions towards which i am advancing, gives me a foretaste of those icy climes. inspirited by this wind of promise, my daydreams become more fervent and vivid. i try in vain to be persuaded that the pole is the seat of frost and desolation; it ever presents itself to my imagination as the region of beauty and delight. there, margaret, the sun is for ever visible, its broad disk just skirting the horizon and diffusing a perpetual splendour. there for wi

In [9]:
with open('./shelley.txt', 'w') as output_file:
    output_file.write(shelley_frank)

In [10]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

bpe_tokenizer = ByteLevelBPETokenizer()

%time bpe_tokenizer.train(files='./shelley.txt', vocab_size=20000, min_frequency=2)




CPU times: user 1.36 s, sys: 441 ms, total: 1.8 s
Wall time: 263 ms


In [32]:
!mkdir ./transf_shelley
model_name = './transf_shelley'
bpe_tokenizer.save_model('./transf_shelley')

mkdir: cannot create directory ‘./transf_shelley’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['./transf_shelley/vocab.json', './transf_shelley/merges.txt']

In [28]:
import torch
use_gpu = -1
if torch.cuda.is_available():
    use_gpu = 0

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": './shelley.txt'})
#model_checkpoint = 'stabilityai/stablelm-zephyr-3b'
model_checkpoint = model_name

In [30]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1
    })
})


In [31]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

ValueError: Unrecognized model in ./transf_shelley. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_conformer, flaubert, flava, fnet, focalnet, fsmt, funnel, fuyu, gemma, 

In [ ]:
def tokenize(examples):
    return tokenizer(examples['text'])

In [ ]:
tokenized_data = datasets.map(tokenize, batched = True, num_proc = 1, remove_columns = ['text'])

In [ ]:
tokenized_data['train'][0]

In [ ]:
# Hyperparams
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_data.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

tokenizer.decode(lm_datasets["train"][1]["input_ids"])

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-test",
     per_device_train_batch_size=2,  # Lower the batch size
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch
    #evaluation_strategy = "epoch",
    num_train_epochs = 20,
    learning_rate=2e-5,
    weight_decay=0.01,
    #push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    #eval_dataset=lm_datasets["validation"],
)


In [ ]:
trainer.save_model('tuned-shelley')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

s_model = AutoModelForCausalLM.from_pretrained("tuned-shelley")
s_tokenizer = AutoTokenizer.from_pretrained("./shelley_safe")


In [ ]:
#Txt gen attempr

prompt = 'you will rejoice'

input_ids = s_tokenizer(prompt, return_tensors='pt').input_ids

outputs = s_model.generate(
    input_ids,
    max_length=512,           # Maximum length of generated text
    num_return_sequences=1,  # Number of generated sequences
    do_sample=True,          # Use sampling instead of greedy decoding
    top_k=5,                # Use top-k sampling
    #top_p=0.95,              # Use nucleus sampling
    temperature=1.0,         # Control randomness
)


generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)